# Ансамбли моделей машинного обучения. Часть 2


## Описание датасета

Для лабортаторной работы был выбран датасет California Housing Prices:


## Импорт библиотек


In [6]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Загрузка и перевичный анализ данных


In [7]:
# Загрузка данных
data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

# Проверка на наличие пропусков
print(X.isnull().sum())

# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Объединение признаков и целевой переменной в один DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

# Вывод первых 5 строк объединенного DataFrame
print(df.head())

MedInc        0
HouseAge      0
AveRooms      0
AveBedrms     0
Population    0
AveOccup      0
Latitude      0
Longitude     0
dtype: int64
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.0  2.109842     37.86   
2  7.2574      52.0  8.288136   1.073446       496.0  2.802260     37.85   
3  5.6431      52.0  5.817352   1.073059       558.0  2.547945     37.85   
4  3.8462      52.0  6.281853   1.081081       565.0  2.181467     37.85   

   Longitude  target  
0    -122.23   4.526  
1    -122.22   3.585  
2    -122.24   3.521  
3    -122.25   3.413  
4    -122.25   3.422  


## Разделение выборки на обучающую и тестовую


In [8]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Обучение моделей


### Модель стекинга


In [9]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

# Определение базовых моделей
estimators = [
    ('ridge', RidgeCV()),
    ('dt', DecisionTreeRegressor(random_state=42)),
    ('svr', SVR())
]

# Модель стекинга
stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=LinearRegression()
)

# Обучение модели
stacking_model.fit(X_train, y_train)


StackingRegressor(estimators=[('ridge', RidgeCV()),
                              ('dt', DecisionTreeRegressor(random_state=42)),
                              ('svr', SVR())],
                  final_estimator=LinearRegression())

### Модель многослойного персептрона


In [10]:
from sklearn.neural_network import MLPRegressor

# Определение модели
mlp_model = MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42)

# Обучение модели
mlp_model.fit(X_train, y_train)


MLPRegressor(max_iter=500, random_state=42)

## Оценка качества моделей

Для оценки качества моделей будем использовать метрику **Mean Squared Error (MSE)**


In [22]:
from sklearn.metrics import mean_squared_error

# Оценка модели стекинга
stacking_pred = stacking_model.predict(X_test)
stacking_mse = mean_squared_error(y_test, stacking_pred)

# Оценка модели многослойного персептрона
mlp_pred = mlp_model.predict(X_test)
mlp_mse = mean_squared_error(y_test, mlp_pred)

# Вывод результатов
print(f'Stacking MSE: {stacking_mse}')
print(f'MLP MSE: {mlp_mse}')


Stacking MSE: 0.30420037771556646
MLP MSE: 0.2993013569932049


## Сравнение качества моделей


In [23]:
results = {
    'Stacking': stacking_mse,
    'MLP\t': mlp_mse,
}

# Сравнение
for model, mse in results.items():
    print(f'{model} model MSE: {mse}')

Stacking model MSE: 0.30420037771556646
MLP	 model MSE: 0.2993013569932049
